In [4]:
import pandas as pd
import pbipy
import requests
from azure.identity import ClientSecretCredential
import json

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
tenant_id = os.getenv('tenant_id')
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')

In [6]:
workspace_name = 'dev-test-gj'

In [7]:
pbi_api = 'https://analysis.windows.net/powerbi/api/.default'

In [8]:
auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                                            tenant_id = tenant_id,
                                            client_id = client_id,
                                            client_secret = client_secret)
access_token = auth.get_token(pbi_api)
access_token = access_token.token

In [10]:
base_url = 'https://api.powerbi.com/v1.0/myorg/'
header = {'Authorization': f'Bearer {access_token}',
          'Content-Type': 'application/json'}

In [11]:
pbi = pbipy.PowerBI(access_token)

In [12]:
admin = pbi.admin()

In [13]:
def check_workspace_exists(df, workspace_name):
    existing_name = set(df['name'].values)
    result = workspace_name in existing_name
    return result

In [14]:
workspace_tenant = admin.groups()

In [16]:
data = [{'Group id': ws.id, 'name': ws.name} for ws in workspace_tenant]

In [18]:
df_workspace = pd.DataFrame(data)

In [20]:
workspace_check_existence = check_workspace_exists(df_workspace, workspace_name)

In [22]:
if not workspace_check_existence:
    # Create DEV workspace
    workspace_dev = pbi.create_group(name = workspace_name)
    print(f"Workspace '{workspace_dev.name}' created.")
    # Create TEST workspace
    workspace_test = pbi.create_group(name = workspace_name.replace('dev', 'test'))
    print(f"Workspace '{workspace_test.name}' created.")
    # Create PROD workspace
    workspace_prod = pbi.create_group(name = workspace_name.replace('dev', 'prod'))
    print(f"Workspace '{workspace_prod.name}' created.")
else:
    print(f"A workspace with the name '{workspace_name}' already exists")

Workspace 'dev-test-gj' created.
Workspace 'test-test-gj' created.
Workspace 'prod-test-gj' created.
